In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
import random
import string
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical

In [2]:
vocab_sizeee=1000
batch_size=32
max_len=15

In [3]:
data=pd.read_csv("F:/open_zip/flickr1k/captions.csv")

In [4]:
data.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [5]:
data["caption"]=data["caption"].apply(lambda line:word_tokenize(line.lower()))

In [6]:
data.head()

,image,caption
0,1000268201_693b08cb0e.jpg,"[a, child, in, a, pink, dress, is, climbing, u..."
1,1000268201_693b08cb0e.jpg,"[a, girl, going, into, a, wooden, building, .]"
2,1000268201_693b08cb0e.jpg,"[a, little, girl, climbing, into, a, wooden, p..."
3,1000268201_693b08cb0e.jpg,"[a, little, girl, climbing, the, stairs, to, h..."
4,1000268201_693b08cb0e.jpg,"[a, little, girl, in, a, pink, dress, going, i..."


In [8]:
words=[]

for line in data["caption"]:
    words.extend(line)

In [9]:
len(words)

59960

In [10]:
wordss,count=np.unique(words,return_counts=True)

In [11]:
len(wordss)

3290

In [12]:
vocab=wordss[np.argsort(count)[-vocab_sizeee:]]

In [13]:
len(vocab)

1000

In [14]:
bad=stopwords.words("english") + list(string.punctuation)

In [15]:
len(bad)

211

In [16]:
vocab_size=[word for word in vocab if word not in bad]

In [17]:
vocab_size=len(vocab_size)

In [18]:
vocab_size

927

In [19]:
vocab=dict(zip(vocab,np.arange(1,vocab_size+1)))

In [20]:
rev_vocab=dict(zip(np.arange(1,vocab_size+1),vocab))

In [21]:
sent=data.caption[2000]

In [22]:
" ".join(sent[1:-1])

'climber in an orange helmet is ascending attached to a rope whilst climbing a rock face'

In [23]:
" ".join(word for word in sent[1:-1] if word in vocab)

'climber orange helmet attached rope whilst face'

In [24]:
#line = random.choice(data.caption)
#line = [vocab[word] for word in line if word in vocab]

#line

In [28]:
def data_gen(lines,image, batch_size=32):
    
    n = 0
    
    X1,X2, Y = [], [],[]
    
    while True:
        
        index=random.randint(1,len(image)-1)
        img=image[index]
        Image=np.array(load_img("F:/open_zip/flickr1k/images/"+img ,target_size=(255,255,3)))
        
        line = lines[index]
        line = [vocab[word] for word in line if word in vocab]
        if len(line) > 4:
            index = random.randint(0, len(line)-1)
            X1.append(Image)
            X2.append(line[:index])
            Y.append(to_categorical(line[index], num_classes=vocab_size+1))
            n += 1

            if n % batch_size == 0:
                X_mod1=np.array(X1)
                X_mod2 = pad_sequences(X2, maxlen=max_len)
                y_mod = np.array(Y)
                X1,X2, Y = [], [],[]
                yield [[X_mod1,X_mod2], y_mod]


In [29]:
dat=data_gen(data["caption"],data["image"])

In [30]:
datas=next(dat)

In [37]:
datas[0][1].shape
#b=datas[1][0]

(32, 15)

In [51]:
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense,Flatten,concatenate
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Model

In [43]:
resnet=ResNet50(include_top=False,weights="imagenet",input_shape=(255,255,3))

In [44]:
for layer in resnet.layers:
    layer.trainable=False

In [46]:
in_layer=Input(shape=(max_len,))
embad=Embedding(input_dim=vocab_size+1 , output_dim=10)(in_layer)
rnn=LSTM(units=500)(embad)
encoded_caption=Dense(units=256,activation="tanh")(rnn)

In [49]:
flat=Flatten()(resnet.output)
encoded_image= Dense(256, activation="relu")(flat)

In [53]:
out_layer = Dense(units=vocab_size+1, activation="softmax")(concat)

In [54]:
model=Model(inputs=[in_layer,resnet.input],outputs=out_layer)

In [55]:
model.compile(    optimizer='rmsprop',
    loss="categorical_crossentropy")

In [ ]:
gen=data_gen(data.caption,data.image,batch_size=2000)


In [44]:
sel=data["caption"][125][:8]

In [45]:
sel

['a', 'boy', 'with', 'a', 'stick', 'kneeling', 'in', 'front']

In [46]:
final=[vocab[w] for w in sel if w in vocab]

In [47]:
final

[872, 57]

In [48]:
X=pad_sequences([final],maxlen=max_len)

In [49]:
X

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        872,  57]])

In [50]:
model.predict(X).argmax(axis=1)

array([911], dtype=int64)

In [51]:
rev_vocab[911]

'as'